## Explore Board Representations

In [1]:
import numpy as np
import backgammon as bg

## Ideas
* cribbing this from [pgx](https://github.com/sotetsuk/pgx/blob/main/docs/backgammon.md)
* there are 24 points, you represent pieces by 1 for white and -1 for black
* you add to the point you move to, you subtract to the point you take away
* you evaluate legal moves by checking to see that >= -1 for white, <= 1 for black
* say white goes 0-23, black goes 23-0

In [2]:
board = np.zeros([1,24],dtype=int)

In [3]:
board

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]])

In [4]:
board.reshape([4,6])

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [5]:
initial_board = bg.create_initial_board()

In [6]:
initial_board

array([ 2,  0,  0,  0,  0, -5,  0, -3,  0,  0,  0,  5, -5,  0,  0,  0,  3,
        0,  5,  0,  0,  0,  0, -2])

In [7]:
test_roll = bg.roll()
test_roll

array([2, 3])

## Translate a 24 value numpy array into a visualized board state 

In [8]:
bg.draw_board(initial_board)


     |---------------------|
  12 |XXXXX     |     OOOOO|13
  11 |          |          |14
  10 |          |          |15
   9 |          |          |16
   8 |OOO       |       XXX|17
   7 |          |          |18
     |---------------------|
     |---------------------|
   6 |OOOOO     |     XXXXX|19
   5 |          |          |20
   4 |          |          |21
   3 |          |          |22
   2 |          |          |23
   1 |XX        |        OO|24
     |---------------------|

    X remaining pips : 152
    O remaining pips : 152
    


## Try rolling 3/1 from the initial board

In [9]:
initial_board = bg.create_initial_board()
new_position = bg.move(position=initial_board, 
                    roll=[1,3], 
                    move='6/5 8/5', 
                    pieces='x')

In [10]:
bg.draw_board(new_position)


     |---------------------|
  12 |XXXXX     |     OOOOO|13
  11 |          |          |14
  10 |          |          |15
   9 |          |          |16
   8 |OOO       |        XX|17
   7 |          |          |18
     |---------------------|
     |---------------------|
   6 |OOOOO     |      XXXX|19
   5 |          |        XX|20
   4 |          |          |21
   3 |          |          |22
   2 |          |          |23
   1 |XX        |        OO|24
     |---------------------|

    X remaining pips : 148
    O remaining pips : 152
    


In [11]:
new_position2 = bg.move(position=new_position, 
                    roll=[4,2], 
                    move='8/4 6/4', 
                    pieces='o')

In [12]:
bg.draw_board(new_position2)


     |---------------------|
  12 |XXXXX     |     OOOOO|13
  11 |          |          |14
  10 |          |          |15
   9 |          |          |16
   8 |OO        |        XX|17
   7 |          |          |18
     |---------------------|
     |---------------------|
   6 |OOOO      |      XXXX|19
   5 |          |        XX|20
   4 |OO        |          |21
   3 |          |          |22
   2 |          |          |23
   1 |XX        |        OO|24
     |---------------------|

    X remaining pips : 148
    O remaining pips : 146
    


In [13]:
new_position_3 = bg.move(position=new_position2, 
                    roll=[5,6], 
                    move='24/13', 
                    pieces='x') 

This is not a legal move


## :-(

## TODO
- somehow initial board gets updated during this process as well
- doesnt properly handle moving the same piece twice
- [X] add the ability to do that
- [X] add the ability to write the above like '24/13'

## Next up
- [ ] add bar
- [ ] add hitting
- [ ] add bearing off
- [ ] refactor the game state into a class

In [14]:
class Backgammon:
        
    def __init__(self):
        self.position = bg.create_initial_board()
        self.bar = []

    def roll(self):
        return np.random.randint(low=1, high=6, size=2)
    
    def create_checkers(self, position):
        drawn_points = []
        for point in position[:12]:
            if point == 0:
                drawn_point = 10 * ' '
            if point >= 0:
                drawn_point = 'X' * point + (10 - point) * ' '
            if point < 0:
                drawn_point = 'O' * abs(point) + (10 + point) * ' '
            drawn_points.append(drawn_point)
        for point in position[12:]:
            if point == 0:
                drawn_point = 10 * ' '
            if point >= 0:
                drawn_point = (10 - point) * ' ' + 'X' * point 
            if point < 0:
                drawn_point = (10 + point) * ' ' + 'O' * abs(point)  
            drawn_points.append(drawn_point)
        return drawn_points

    def calculate_remaining_pips(self, position):
        x_pips = o_pips = 0
        for i, point in enumerate(position):
            if point > 0:
                dist = 23 - i
                x_pips += point * dist
            if point < 0:
                dist = i
                o_pips += point * dist * -1
        return x_pips, o_pips

    def draw_board(self, position):
        drawn_position = create_checkers(position) 
        x_pips, o_pips = calculate_remaining_pips(position)
        
        print(f'''
         |---------------------|
      12 |{drawn_position[11]}|{drawn_position[12]}|13
      11 |{drawn_position[10]}|{drawn_position[13]}|14
      10 |{drawn_position[9]}|{drawn_position[14]}|15
       9 |{drawn_position[8]}|{drawn_position[15]}|16
       8 |{drawn_position[7]}|{drawn_position[16]}|17
       7 |{drawn_position[6]}|{drawn_position[17]}|18
         |---------------------|
         |---------------------|
       6 |{drawn_position[5]}|{drawn_position[18]}|19
       5 |{drawn_position[4]}|{drawn_position[19]}|20
       4 |{drawn_position[3]}|{drawn_position[20]}|21
       3 |{drawn_position[2]}|{drawn_position[21]}|22
       2 |{drawn_position[1]}|{drawn_position[22]}|23
       1 |{drawn_position[0]}|{drawn_position[23]}|24
         |---------------------|
    
        X remaining pips : {x_pips}
        O remaining pips : {o_pips}
        ''')

        
    def parse_move(self, move, roll):
        checks_moved = move.count('/')
    
        if checks_moved == 2:
            first_move = move.split(' ')[0]
            second_move = move.split(' ')[1]
            
            first_start = int(first_move.split('/')[0])
            first_end = int(first_move.split('/')[1])
            second_start = int(second_move.split('/')[0])
            second_end = int(second_move.split('/')[1])
            #check if it fits the roll
            diff1= abs(first_end - first_start)
            diff2= abs(second_end - second_start)
            if sorted(list(roll)) != sorted([diff1, diff2]):
                print('This is not a legal move because of dice')
                return
        else:
            first_start = int(move.split('/')[0])
            second_end = int(move.split('/')[1])
            diff = first_start - second_end
            first_end = second_start = first_start - roll[0]
            
            if sum(roll) != diff:
                print('This is not a legal move because of dice')
                return           
        #this will fail if you can move a piece 3 and then 5, but not 5 and then 3    
        return [first_start, first_end, second_start, second_end]
    
    def adjust_board_for_move(self, position, pieces, bar=[]):
        'ensure pieces always go down in indices and up in checkers'
        if pieces not in (['x','o']):
            print('Not a legal position')
            return
            
        if pieces == 'x':
            return np.flip(position)
        if pieces == 'o':
            return position * -1
        
    def update_board(self, move,position,pieces='x', bar=[]):
        #initialize board orientation
        adjust_board_for_move(position, pieces)
    
        #decrement the places you're moving from
        position[move[0]] -= 1
        position[move[2]] -= 1
    
        for new_pos in [move[1], move[3]]:
            # when you hit on a loose checker
            if position[new_pos] == -1:
                position[new_pos] = 1
                bar.append('o') if pieces == 'x' else bar.append('x') 
            #otherwise
            else:
                position[new_pos] += 1
    
        # return board orientation to original position
        adjust_board_for_move(position, pieces)
    
        return position
        
    def move(self, position, roll, move, pieces='x'):
        
        move = parse_move(move,roll)
            
        #adjust values for indices
        move = [x-1 for x in move]
        position = adjust_board_for_move(position, pieces)
    
        #check where you have checkers and where you can move them
        legal_starts = np.where(position > 0)[0]
        open_spaces = np.where(position >= -1)[0]
        # print(f'legal starts : {legal_starts}')
        # print(set([move[1],move[3]]))
        # print(f'open spaces : {open_spaces}')
        # print(set([move[0],move[2]]))
    
        # confirm these match
        if (set([move[0],move[2]]).issubset(legal_starts)) & set([move[1],move[3]]).issubset(open_spaces):
            new_position = update_board(move,position,pieces)
            # return board orientation to original position
            new_position = adjust_board_for_move(new_position, pieces)
            return new_position
        else:
            print('This is not a legal move')
            return
    
    def play(self):
        for turn in range(3): 
            self.draw_board(self.position)
            roll = self.roll()
            print("Roll:", roll)
            move = input("Enter your move: ")
            self.position = self.move(self.position, roll, move)
            self.draw_board(self.position)
            # Need to add evaluation for game_over


In [15]:
game = Backgammon()